In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from bnlp import BasicTokenizer
tokenizer = BasicTokenizer()

In [ ]:
def create_corpus(texts):
    corpus=[]

    for txt in texts:
        tokens = tokenizer.tokenize(txt)
        corpus.extend(tokens)

    return corpus

In [ ]:
from collections import defaultdict
from collections import  Counter
from bnlp.corpus import stopwords
import matplotlib.font_manager as fm

def show_stop_words(corpus, length=20):
  dic = defaultdict(int)

  for word in corpus:
      if word in stopwords:
          dic[word]+=1

  top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:length]
  x,y=zip(*top)
  print(x)
  print(y)
  plt.figure(figsize=(10,5))
  plt.bar(x,y)
  plt.show()

In [ ]:
from bnlp.corpus import stopwords, punctuations, digits

def filters(corpus):
    res = []
    for i in corpus:
        if i in stopwords:
            continue

        if i in punctuations + '‘' + '’':
            continue
        if i in digits:
            continue

        res.append(i)

    return res

In [ ]:
def get_top_words(corpus):
    dic = defaultdict(int)

    for word in corpus:
        dic[word] +=1
    top = sorted(dic.items(), key=lambda x:x[1],reverse=True)
    x,y=zip(*top)
    return x, y

# Imports

In [1]:
import pandas as pd
train_set = pd.read_csv("stop_words_removed_train_set_2.csv")
test_set = pd.read_csv("test_set_2.csv")
validate_set = pd.read_csv("test_set_1.csv")

In [2]:
import time
from transformers import AutoModel, BertTokenizerFast
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [ ]:
train_list = train_set.values.tolist()

In [ ]:
# print(x)
p = ['একটা','হয়ে','কথা','মানুষ','এক','দিয়ে','সাথে','যায়','হয়েছে',
     'মত','টা','একজন','বড়','বাংলাদেশ','সময়','মানুষের','সবাই','ভিডিও',
     'টাকা','দেশের','একটু','লাগে','মন','দেশে','আল্লাহ','গুলো','বছর','বাংলাদেশের',
     'এসব','বাবা','ছবি','জানি','আসে','দেখি','আরো','পড়ে','চোখ',
     'মাঝে','কারণে','উঠে','ছাড়া','এখনো','এভাবে','কমেন্ট','আজকে','দিলে','—','যাওয়ার',
     'সোমবার','ক্যাম্পাসে','কিনা','জাতীয়','গতকাল','পড়তে','আমিন','শব্দ','হয়ত','করবো',
     'ইচ্ছা','কিনা','এলো','ঈদ','ঢাকার','চলছে','অনুষ্ঠান','ভাইয়ের','বৃষ্টি','আসার','তাদেরকে','রইল',
     'আমিও','ফেসবুক','দিয়েছেন','পরিবেশ','এইরকম','নেওয়ার','আব্বু','দক্ষিণ','সাদা',
     'অনুষ্ঠিত','দিব','উঠছে','এতটা','করল','নাম্বার','সমাজে','বিয়েতে','লাগল','মো','চট্টগ্রামের',
     'শরীরে','রাজধানী','করলেও','মেয়ের','প্রথমে','পূরণ','লাইক','আপুর','পারো','ম্যাচের',
     'মানায়','নিকট','দেহ','পারতাম','পারতো','পত্রিকা','পারভেজ','আলম','ছাত্রীকে','অামি',
     'বাচ্চারা','এমনই','হলুদ','মানবতা','পুরোপুরি','হয়তবা','বাতাস', 'হও', 'কাহিনী','ফেরার',
     'ছয়', 'মিয়ার', 'আসেনা','বাস্তব', 'অর্জন', 'বৃদ্ধ', 'মতন', 'খেলতে', 'বানাতে', 'লাগিয়ে',
     'ছবিও', 'মাংস', 'মেঝেতে','আমিই','দেখেছিলাম','রং','ভাবেননি','দিন','ছবি','কিন্তু','শুরু','অথচ',
     'রাতে','যারা','মনে','এখনে','মানুষ','আমাকে','কথা','গেছে','শুরু','আমিও','করে','পুরো','আমাদের',
     'দিলো','সেই','যে','তবে','টা','তার ','এই ','হচ্ছে ','ছিল ','অনেক ','খুবি ','এর ','বলেন ','পেলে',
     'বয়সের', 'কে', 'হয়','হয়ে'
     ]
n=set(p)
print(len(n))

In [ ]:
for i in range(len(train_list)):
    for j in n:
        if " "+j+" " in train_list[i][0]:
            train_list[i][0]=train_list[i][0].replace(j,"")
        elif j+" " in train_list[i][0]:
            train_list[i][0]=train_list[i][0].replace(j,"")
        elif " "+j in train_list[i][0]:
            train_list[i][0]=train_list[i][0].replace(j,"")
        else:
            continue

In [ ]:
train_set = pd.DataFrame(train_list, columns = ['text', 'label'])


# Optoimizing dataset

In [3]:
class makeDataset(Dataset):
    def __init__(self, data, max_length=100):
        self.data = data
        
        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['text'], value['label']

# Training data 2 testing data 2 final testing data 1

In [4]:
training_data = makeDataset(train_set)
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)

In [5]:
test_data = makeDataset(test_set)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [6]:
final_test_data = makeDataset(validate_set)
final_test_dataloader = DataLoader(final_test_data, batch_size=16, shuffle=True)

In [7]:
class newEmotionBert(nn.Module):

    def __init__(self, bert):
        super(newEmotionBert, self).__init__()

        self.bert = bert
        
        self.embedding = nn.EmbeddingBag(15000, 500, sparse=True)

        # dropout layer
        self.dropout = nn.Dropout(0.2)

        # relu activation function
        self.relu = nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768, 328)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(328, 5)  
        self.softmax = nn.LogSoftmax(dim=1)
#         self.init_weights()
#     def init_weights(self):
#         initrange = .5
#         self.embedding.weight.data.uniform_(-initrange, initrange)
#         self.fc1.weight.data.uniform_(-initrange, initrange)
#         self.fc1.bias.data.zero_()
#         self.fc2.weight.data.uniform_(-initrange, initrange)
#         self.fc2.bias.data.zero_()

    # define the forward pass
    def forward(self, input_ids, token_type_ids, attention_mask):
        # pass the inputs to the model
        out = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        x = self.fc1(out[1])
        x = self.relu(x)
        # output layer
        x = self.fc2(self.dropout(x))
        
        x = self.softmax(x)
        return x

In [8]:
bert_model_name = "sagorsarker/bangla-bert-base"
bert = BertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = newEmotionBert(bert)
model.to(device);

In [10]:
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = nn.NLLLoss()
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

In [11]:
from time import sleep
def train(model, dataloader, optimizer, criterion, config):
    model.train()  # prep model for training
    train_loss = 0
    for batch in tqdm(dataloader):
        text, labels = batch

        model.zero_grad()

        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        labels = labels.to(device)

        # move things to model
        logs = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(logs, labels)
        train_loss += loss.item() * input_ids.size(0)
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    return train_loss

In [12]:
def evaluate(model, dataloader, criterion, config):
    total = 0
    correct = 0
    valid_loss = 0.0

    model.eval()  # prep model for evaluation
    for batch in tqdm(dataloader):
        text, labels = batch
        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        labels = labels.to(device)

        # move things to model
        output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss_p = criterion(output, labels)
        # update running validation loss
        valid_loss += loss_p.item() * input_ids.size(0)
        # calculate accuracy
        proba = torch.exp(output)
        top_p, top_class = proba.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        # accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total, correct, valid_loss

In [13]:
epochs = 5
tokenizer_config = {
    "max_length": 100,
    "padding": "max_length",
    "return_tensors": "pt",
    "truncation": True,
    "add_special_tokens": True
}

In [14]:
train_loss_data, valid_loss_data = [], []
valid_loss_min = np.Inf
since = time.time()
best_loss = np.inf

for epoch in range(epochs):
    print("Epoch: {}/{}".format(epoch + 1, epochs))
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    total = 0
    correct = 0
    e_since = time.time()

    # Train Model
    train_loss += train(model, train_dataloader, optimizer, criterion, tokenizer_config)

    # Now Evaluate
    out = evaluate(model, test_dataloader, criterion, tokenizer_config)

    total += out[0]
    correct += out[1]
    valid_loss += out[2]

    scheduler.step()

    # print training/validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_dataloader.dataset)
    valid_loss = valid_loss / len(test_dataloader.dataset)

    # calculate train loss and running loss
    train_loss_data.append(train_loss * 100)
    valid_loss_data.append(valid_loss * 100)
    
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), "new_emotion_model.pth")

    print("\tTrain loss:{:.6f}..".format(train_loss),
          "\tValid Loss:{:.6f}..".format(valid_loss),
          "\tAccuracy: {:.4f}".format(correct / total * 100))

time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Epoch: 1/5


  0%|          | 0/233 [00:00<?, ?it/s]

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
model.load_state_dict(torch.load("./new_emotion_model.pth", map_location = device))

In [ ]:
all_preds = []
all_labels = []

for batch in final_test_dataloader:
    text, labels = batch
    inputs = tokenizer.batch_encode_plus(
        text, **tokenizer_config
    )
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    labels = labels.to(device)

    # move things to model
    output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)
    preds = output.detach().cpu().numpy()
    preds = np.argmax(preds, axis = 1)
    all_preds.extend(preds)
    all_labels.extend(labels.cpu().numpy())

In [ ]:
correct = 0
for i in range(len(all_labels)):
    if all_labels[i]==all_preds[i]:
        correct+=1
print(correct)

In [ ]:
print(f"accuracy: {correct/len(all_labels)*100}")

In [ ]:
print(bert.embeddings.word_embeddings.weight[101])

In [ ]:
print(bert.embeddings.word_embeddings.weight[10])

In [ ]:
print(bert.embeddings.word_embeddings.weight)

In [ ]:
in_correct = []
for i in range(len(all_labels)):
    if all_labels[i]!=all_preds[i]:
        in_correct.append(train_set["text"][i])
print(in_correct)

In [ ]:
len(in_correct)

In [ ]:
position = []
for i in range(len(all_labels)):
    if all_labels[i]!=all_preds[i]:
        position.append([i])
print(position)

In [ ]:

for i in range(len(all_labels)):
    if all_labels[i]!=all_preds[i]:
        print(f"{all_labels[i]} and {all_preds[i]}")

In [ ]:
wrong = []
for i in position:
    wrong.append(validate_set["text"][i[0]])

In [ ]:
print(wrong)

In [ ]:
# train_set = pd.DataFrame(train_list, columns = ['a','text', 'label'])
val = create_corpus(wrong)
val_filtered = filters(val)
x, y = get_top_words(val_filtered)

In [ ]:
print(x)

In [ ]:
len(x)